This code follows the same steps to test PRS Model

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

# Scikit-learn imports
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, 
    matthews_corrcoef, 
    accuracy_score,
    classification_report, 
    confusion_matrix,
    roc_curve, 
    precision_recall_curve, 
    auc
)

# Imbalanced-learn imports
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

In [ ]:

def load_filtered_data():
    """Load data with explicit dtype specification"""
    # Load with explicit dtype for problematic columns
    df = pd.read_csv("data_participant.csv", low_memory=False)
    demo = pd.read_csv("data_year_of_birth_and_ethnicity.csv", low_memory=False)
    prs = pd.read_csv("prs.csv")
    
    # Rest of your existing function...
    
    # Ensure consistent ID format
    df['eid'] = df['eid'].astype(str).str.strip()
    demo['eid'] = demo['eid'].astype(str).str.strip()
    prs['eid'] = prs['eid'].astype(str).str.strip()
    
    # Merge datasets
    df = df.merge(
        demo[['eid', 'p21022', 'p31', 'p40001_i1', 'p40006_i0', 'p40008_i0']],
        on='eid',
        how='left'
    ).merge(
        prs,  # Merge with PRS data
        on='eid',
        how='left'
    )
    
    # Apply filters
    df = df[
        (df['p31'] != 'Female') &                # Only males
        (df['p21022'] > 50) &                    # Age at recruitment >50
        ~(                                       # Exclude prostate cancer ≤50
            (df['p40006_i0'].astype(str).str.contains("C61", na=False)) & 
            (df['p40008_i0'] <= 50)
        )
    ].copy()
    
    # Create target
    df['target'] = (
        df['p40001_i0'].str.contains('C61', na=False) |
        df['p40001_i1'].str.contains('C61', na=False)
    ).astype(int)
    
    # Remove leakage features
    leakage_features = [
        'p40000_i0', 'p40005_i8', 'p40006_i0', 'p40008_i0',
        *[col for col in df.columns if 'date_of_death' in col.lower()]
    ]
    df = df.drop(columns=[col for col in leakage_features if col in df.columns])
    
    return df

# ==================== FEATURE ENGINEERING ====================
def extract_all_features(df):
    """Feature extraction focusing on specific features and PRS columns"""
    # 1. List of your specified features (without _dur)
    specified_features = [
        'p30850_i0', 'p30720_i0', 'p30700_i0', 
        'p30630_i0', 'p30610_i0', 'p48_i0', 
        'p30830_i0', 'p21002_i0'
    ]
    
    # 2. Your PRS columns (add your specific PRS columns here)
    prs_cols = ['p26267', 'p26268']  # ← REPLACE WITH YOUR ACTUAL PRS COLUMNS
    
    # 3. Other standard features you want to keep
    numerical = ['p21001_i0', 'p50_i0', 'p47_i0']  # example - adjust as needed
    categorical = ['p20116_i0', 'p1618_i0', 'p3637_i0']  # example - adjust as needed
    
    # Create preprocessing pipeline
    numeric_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    categorical_transformer = Pipeline([
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    
    # Combine all numeric features
    all_numeric = specified_features + numerical + prs_cols
    
    preprocessor = ColumnTransformer([
        ('num', numeric_transformer, all_numeric),
        ('cat', categorical_transformer, categorical)
    ])
    
    # Apply preprocessing
    X = df.drop(columns=['target'])
    y = df['target']
    X_processed = preprocessor.fit_transform(X)
    
    # Get feature names
    categorical_features = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical)
    feature_names = all_numeric + list(categorical_features)
    
    return X_processed, y, feature_names


def training_split(x, y, test_size=0.2, random_state=42):
    """
    Splits the data into train and test sets, ensuring no NaNs in y.
    
    Parameters:
    - x: pd.DataFrame or np.ndarray of features
    - y: pd.Series or np.ndarray of labels
    - test_size: proportion of data to use as test set
    - random_state: random seed for reproducibility
    
    Returns:
    - X_train, X_test, y_train, y_test
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    
    mask = ~y.isna()
    x = x[mask]
    y = y[mask]
    
    X_train, X_test, y_train, y_test = train_test_split(
        x, y,
        test_size=test_size,
        random_state=random_state,
        shuffle=True,
        stratify=y
    )
    
    return X_train, X_test, y_train, y_test




def retrain_logreg_top_features_test(
    X_train, y_train, X_test, y_test, feature_names,
    specified_features=None,
    prs_features=['p26267', 'p26268'],
    n_features=24,
    output_dir="output",
    eids_test=None,
    random_state=42
):
    """
    Complete and fixed logistic regression training with:
    - Feature selection
    - SMOTE balancing
    - Comprehensive evaluation
    - PRS analysis
    - High-quality visualizations
    """
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. Feature Selection
    if specified_features is None:
        specified_features = [
            'p30850_i0', 'p30720_i0', 'p30700_i0',
            'p30630_i0', 'p30610_i0', 'p48_i0',
            'p30830_i0', 'p21002_i0'
        ]
    
    # Get available features
    available_features = [f for f in specified_features + prs_features if f in feature_names]
    selected_features = available_features[:n_features]
    print(f"Using {len(selected_features)} features including {len([f for f in selected_features if f in prs_features])} PRS features")
    
    # 2. Data Preparation
    if isinstance(X_train, np.ndarray):
        feat_idx = [feature_names.index(f) for f in selected_features]
        X_train_sub = X_train[:, feat_idx]
        X_test_sub = X_test[:, feat_idx]
    else:
        X_train_sub = X_train[selected_features]
        X_test_sub = X_test[selected_features]
    
    # 3. Apply SMOTE
    sm = SMOTE(random_state=random_state)
    X_bal, y_bal = sm.fit_resample(X_train_sub, y_train)
    print(f"Class balance after SMOTE: {sum(y_bal)} positives, {len(y_bal)-sum(y_bal)} negatives")
    
    # 4. Model Training
    model = LogisticRegression(
        max_iter=1000,
        solver='liblinear',
        class_weight='balanced',
        random_state=random_state
    )
    model.fit(X_bal, y_bal)
    
    # ======================
    # 5. Plotting Functions
    # ======================
    
    def plot_combined_curves(y_true, probs, split_name):
        """Plot ROC and PR curves side by side"""
        plt.figure(figsize=(12, 5))
        
        # ROC Curve
        plt.subplot(1, 2, 1)
        fpr, tpr, _ = roc_curve(y_true, probs)
        roc_auc = roc_auc_score(y_true, probs)
        plt.plot(fpr, tpr, color='blue', lw=2, 
                label=f'ROC (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'{split_name} ROC Curve')
        plt.legend(loc="lower right")
        
        # PR Curve
        plt.subplot(1, 2, 2)
        precision, recall, _ = precision_recall_curve(y_true, probs)
        pr_auc = auc(recall, precision)
        plt.plot(recall, precision, color='green', lw=2,
                label=f'PR (AUC = {pr_auc:.2f})')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'{split_name} Precision-Recall Curve')
        plt.legend(loc="lower left")
        
        plt.tight_layout()
        plt.savefig(f"{output_dir}/{split_name}_curves.png", dpi=300)
        plt.close()
    
    def plot_feature_importance(coefs, features):
        """Plot horizontal bar chart of feature coefficients"""
        plt.figure(figsize=(10, 6))
        feat_imp = pd.DataFrame({
            'Feature': features, 
            'Coefficient': coefs,
            'Absolute': np.abs(coefs)
        }).sort_values('Absolute', ascending=True)
        
        colors = ['red' if x in prs_features else 'blue' for x in feat_imp.Feature]
        plt.barh(feat_imp['Feature'], feat_imp['Coefficient'], color=colors)
        plt.title('Feature Coefficients (PRS in Red)')
        plt.xlabel('Coefficient Value')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f"{output_dir}/feature_importance.png", dpi=300)
        plt.close()
    
    def plot_confusion_matrix(y_true, preds, split_name, metrics):
        """Plot annotated confusion matrix"""
        cm = confusion_matrix(y_true, preds)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Negative', 'Positive'],
                   yticklabels=['Negative', 'Positive'])
        plt.title(f'{split_name} Confusion Matrix\nAUC: {metrics["auc"]:.2f} | MCC: {metrics["mcc"]:.2f}')
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.tight_layout()
        plt.savefig(f"{output_dir}/{split_name}_confusion_matrix.png", dpi=300)
        plt.close()
    
    # 6. Generate Predictions and Plots
    results = {}
    for split_name, X, y in [('Train', X_train_sub, y_train), 
                            ('Test', X_test_sub, y_test)]:
        probs = model.predict_proba(X)[:, 1]
        preds = (probs >= 0.5).astype(int)
        
        # Calculate metrics
        split_results = {
            'auc': roc_auc_score(y, probs),
            'mcc': matthews_corrcoef(y, preds),
            'accuracy': accuracy_score(y, preds),
            'report': classification_report(y, preds, output_dict=True)
        }
        results[split_name.lower()] = split_results
        
        # Generate plots
        plot_combined_curves(y, probs, split_name)
        plot_confusion_matrix(y, preds, split_name, split_results)
    
    # Feature Importance Plot
    plot_feature_importance(model.coef_[0], selected_features)
    
    # 7. PRS Analysis
    prs_results = {}
    if prs_features:
        for prs in prs_features:
            if prs in selected_features:
                idx = selected_features.index(prs)
                prs_results[prs] = {
                    'coefficient': model.coef_[0][idx],
                    'abs_importance': abs(model.coef_[0][idx]),
                    'rank': np.argsort(np.abs(model.coef_[0]))[::-1].tolist().index(idx) + 1
                }
    
    return {
        'model': model,
        'features': selected_features,
        'results': results,
        'prs_analysis': prs_results,
        'plots': {
            'roc_pr_curves': f"{output_dir}/Test_curves.png",
            'confusion_matrix': f"{output_dir}/Test_confusion_matrix.png",
            'feature_importance': f"{output_dir}/feature_importance.png"
        }
    }

In [3]:
print("Loading and preprocessing data...")
df = load_filtered_data()


# 2. Extract features and target
print("Extracting features...")
X, y, feature_names = extract_all_features(df)

Loading and preprocessing data...
Extracting features...


In [4]:
print("Splitting data into train/test sets...")
X_train, X_test, y_train, y_test = training_split(X, y)

results = retrain_logreg_top_features_test(
    X_train, y_train, X_test, y_test, feature_names,
    specified_features=['p30850_i0', 'p30720_i0', ...],  # Your features
    prs_features=['p26267', 'p26268'],  # Your PRS columns
    output_dir="./my_results"
)


print(f"Test AUC: {results['results']['test']['auc']}")
print(f"Train MCC: {results['results']['test']['mcc']}")
print(f"PRS coefficients: {results['prs_analysis']}")

# To see all available metrics:
print("\nFull test results:")
for metric, value in results['results']['test'].items():
    print(f"{metric}: {value}")

Splitting data into train/test sets...


Using 4 features including 2 PRS features


d:\bioinfo\fob\pymol\Newfolder\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


Class balance after SMOTE: 134367 positives, 134367 negatives
Test AUC: 0.6840155638149599
Train MCC: 0.05251007906199221
PRS coefficients: {'p26267': {'coefficient': 0.668440373141921, 'abs_importance': 0.668440373141921, 'rank': 1}, 'p26268': {'coefficient': -0.05174845566029922, 'abs_importance': 0.05174845566029922, 'rank': 4}}

Full test results:
auc: 0.6840155638149599
mcc: 0.05251007906199221
accuracy: 0.6407941705755672
report: {'0': {'precision': 0.9947329513953058, 'recall': 0.6409264110502501, 'f1-score': 0.7795640524295749, 'support': 33592.0}, '1': {'precision': 0.015588019260589243, 'recall': 0.6262295081967213, 'f1-score': 0.030418856505813028, 'support': 305.0}, 'accuracy': 0.6407941705755672, 'macro avg': {'precision': 0.5051604853279476, 'recall': 0.6335779596234856, 'f1-score': 0.40499145446769397, 'support': 33897.0}, 'weighted avg': {'precision': 0.9859227556758884, 'recall': 0.6407941705755672, 'f1-score': 0.7728233590125543, 'support': 33897.0}}
